In [20]:
import pandas as pd
df = pd.read_csv("../csv/jago_reviews_raw.csv")
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,lang
0,a22d3997-f03f-4177-b517-2e9182cb02d8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,mantap,5,0,8.82.0,2026-02-27 15:41:08,NaN,NaN,8.82.0,id
1,4ce9e59e-640d-4fe8-b1a1-29680b6d9d88,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Gimana saya masukin kode nya kl mau masukin ko...,1,0,8.82.0,2026-02-27 15:15:43,"Halo, Jagoan. Mohon maaf atas kendala yang ter...",2026-02-27 16:13:28,8.82.0,id
2,835f8aae-03f6-4e2f-931f-c7b416fa1e57,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"tiba"" kantong utama di blokir salah pin gapern...",1,0,8.82.0,2026-02-27 15:13:41,"Halo, Jagoan. Mohon maaf atas kendala yang ter...",2026-02-27 16:17:32,8.82.0,id
3,7c571ea2-3e78-465a-951a-6006d4402ca8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Apk gajelas saldo gopay gua hilang Mulu gara g...,1,0,NaN,2026-02-27 14:14:42,"Halo, Jagoan. Mohon maaf atas kendala yang ter...",2026-02-27 15:25:07,NaN,id
4,0c7277ed-381a-45e1-a1e6-49ab60f1440a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya pengen bank jago bisa memindahkan akun ja...,4,0,NaN,2026-02-27 13:21:33,"Halo, Jagoan! Terima kasih telah menggunakan B...",2026-02-27 14:53:29,NaN,id


In [21]:
df["at"] = pd.to_datetime(df["at"]) #ubah jadi datetime
df = df[["reviewId", "content", "score", "at", "lang"]]

df.info()
df.head()


<class 'pandas.DataFrame'>
RangeIndex: 75851 entries, 0 to 75850
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   reviewId  75851 non-null  str           
 1   content   75851 non-null  str           
 2   score     75851 non-null  int64         
 3   at        75851 non-null  datetime64[us]
 4   lang      75851 non-null  str           
dtypes: datetime64[us](1), int64(1), str(3)
memory usage: 2.9 MB


,reviewId,content,score,at,lang
0,a22d3997-f03f-4177-b517-2e9182cb02d8,mantap,5,2026-02-27 15:41:08,id
1,4ce9e59e-640d-4fe8-b1a1-29680b6d9d88,Gimana saya masukin kode nya kl mau masukin ko...,1,2026-02-27 15:15:43,id
2,835f8aae-03f6-4e2f-931f-c7b416fa1e57,"tiba"" kantong utama di blokir salah pin gapern...",1,2026-02-27 15:13:41,id
3,7c571ea2-3e78-465a-951a-6006d4402ca8,Apk gajelas saldo gopay gua hilang Mulu gara g...,1,2026-02-27 14:14:42,id
4,0c7277ed-381a-45e1-a1e6-49ab60f1440a,saya pengen bank jago bisa memindahkan akun ja...,4,2026-02-27 13:21:33,id


In [22]:
df_id = df[df["lang"] == "id"].copy() #ngambil yang langnya id
print(f"Jumlah review bahasa Indonesia: {len(df_id)}")
df_id["score"].value_counts().sort_index() # buat lihat persebaran untuk bintangnya




Jumlah review bahasa Indonesia: 67355


score
1    16379
2     2556
3     3055
4     3660
5    41705
Name: count, dtype: int64

In [23]:
def label_sentiment(score):
    if score <= 3:
        return "negatif"
    else:
        return "positif"

df_id["sentiment"] = df_id["score"].apply(label_sentiment)
df_id["sentiment"].value_counts()


sentiment
positif    45365
negatif    21990
Name: count, dtype: int64

In [24]:
import re

#Bikin salinan hanya kolom yang dipakai
df_text = df_id[["reviewId", "content", "score", "sentiment"]].copy()

#Ubah ke lowercase
df_text["clean_text"] = df_text["content"].str.lower()

#Hapus URL
df_text["clean_text"] = df_text["clean_text"].apply(
    lambda x: re.sub(r"http\S+|www\S+", "", x)
)

#Hapus karakter non-huruf (angka, simbol, tanda baca berlebihan)
df_text["clean_text"] = df_text["clean_text"].apply(
    lambda x: re.sub(r"[^a-zA-Záéíóúüñçàèìòùâêîôûäëïöü\s]", " ", x)
)

#Hapus spasi berlebih
df_text["clean_text"] = df_text["clean_text"].apply(
    lambda x: re.sub(r"\s+", " ", x).strip()
)


df_text["clean_text"] = df_text["clean_text"].fillna("")

#Lihat hasil beberapa contoh
df_text[["content", "clean_text"]].head(10)


,content,clean_text
0,mantap,mantap
1,Gimana saya masukin kode nya kl mau masukin ko...,gimana saya masukin kode nya kl mau masukin ko...
2,"tiba"" kantong utama di blokir salah pin gapern...",tiba kantong utama di blokir salah pin gaperna...
3,Apk gajelas saldo gopay gua hilang Mulu gara g...,apk gajelas saldo gopay gua hilang mulu gara g...
4,saya pengen bank jago bisa memindahkan akun ja...,saya pengen bank jago bisa memindahkan akun ja...
5,Suka fitur rekening bersama dan bisa integrasi...,suka fitur rekening bersama dan bisa integrasi...
6,"Oke permasalahan sudah selesai, saya naikkan l...",oke permasalahan sudah selesai saya naikkan la...
7,"Bank Jago memang top, gratis biaya admin semua...",bank jago memang top gratis biaya admin semuan...
8,"Akun jago sy diblokir, sampai skrg ga bisa dib...",akun jago sy diblokir sampai skrg ga bisa dibu...
9,keren,keren


In [25]:
import nltk

# Download resources 
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

# hapus stopword
stopwords_id_en = set([
    "dan", "yang", "di", "ke", "dari", "untuk", "ini", "itu", "saya", "aku",
    "the", "is", "am", "are", "to", "in", "on", "of", "for", "this", "that"
])

def preprocess_tokens(text):
    # 1. Tokenisasi
    tokens = word_tokenize(text)
    # 2. Buang stopwords
    tokens = [t for t in tokens if t.lower() not in stopwords_id_en]
    return tokens

# lakuin di kolom clean_text
df_text["tokens"] = df_text["clean_text"].apply(preprocess_tokens)

# Lihat contoh
df_text[["clean_text", "tokens"]].head(10)

[nltk_data] Downloading package punkt to /Users/macos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/macos/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,clean_text,tokens
0,mantap,[mantap]
1,gimana saya masukin kode nya kl mau masukin ko...,"[gimana, masukin, kode, nya, kl, mau, masukin,..."
2,tiba kantong utama di blokir salah pin gaperna...,"[tiba, kantong, utama, blokir, salah, pin, gap..."
3,apk gajelas saldo gopay gua hilang mulu gara g...,"[apk, gajelas, saldo, gopay, gua, hilang, mulu..."
4,saya pengen bank jago bisa memindahkan akun ja...,"[pengen, bank, jago, bisa, memindahkan, akun, ..."
5,suka fitur rekening bersama dan bisa integrasi...,"[suka, fitur, rekening, bersama, bisa, integra..."
6,oke permasalahan sudah selesai saya naikkan la...,"[oke, permasalahan, sudah, selesai, naikkan, l..."
7,bank jago memang top gratis biaya admin semuan...,"[bank, jago, memang, top, gratis, biaya, admin..."
8,akun jago sy diblokir sampai skrg ga bisa dibu...,"[akun, jago, sy, diblokir, sampai, skrg, ga, b..."
9,keren,[keren]


In [ ]:
df_text.to_csv("../csv/jago_preprocessing.csv", index=False)


In [27]:
df_text["sentiment"].value_counts()
df_text.head()


,reviewId,content,score,sentiment,clean_text,tokens
0,a22d3997-f03f-4177-b517-2e9182cb02d8,mantap,5,positif,mantap,[mantap]
1,4ce9e59e-640d-4fe8-b1a1-29680b6d9d88,Gimana saya masukin kode nya kl mau masukin ko...,1,negatif,gimana saya masukin kode nya kl mau masukin ko...,"[gimana, masukin, kode, nya, kl, mau, masukin,..."
2,835f8aae-03f6-4e2f-931f-c7b416fa1e57,"tiba"" kantong utama di blokir salah pin gapern...",1,negatif,tiba kantong utama di blokir salah pin gaperna...,"[tiba, kantong, utama, blokir, salah, pin, gap..."
3,7c571ea2-3e78-465a-951a-6006d4402ca8,Apk gajelas saldo gopay gua hilang Mulu gara g...,1,negatif,apk gajelas saldo gopay gua hilang mulu gara g...,"[apk, gajelas, saldo, gopay, gua, hilang, mulu..."
4,0c7277ed-381a-45e1-a1e6-49ab60f1440a,saya pengen bank jago bisa memindahkan akun ja...,4,positif,saya pengen bank jago bisa memindahkan akun ja...,"[pengen, bank, jago, bisa, memindahkan, akun, ..."
